*Завдання 1: Розбийте MNIST на тренувальну та навчальну вибірки (60k + 10k). Натренуйте Random Forest classifier ти виміряйте час тренування. Обчисліть точність на тестовій вибірці.*

In [8]:
import numpy as np
from sklearn.datasets import fetch_openml


In [9]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Завантаження даних
with open('mnist_data.pkl', 'rb') as f:
    mnist = pickle.load(f)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

# Розбиття на тренувальну (60k) та тестову (10k) вибірки
X_train = X[:60000]
y_train = y[:60000]

X_test = X[60000:]
y_test = y[60000:]

print(f"Тренувальна вибірка: {X_train.shape}")
print(f"Тестова вибірка: {X_test.shape}")

Тренувальна вибірка: (60000, 784)
Тестова вибірка: (10000, 784)


був файл локально з попереднього дз тому так і відкриваю і розбиваю


In [17]:
# Тренування Random Forest
rf_no_pca = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

start = time.time()
rf_no_pca.fit(X_train, y_train)
time_rf_no_pca = time.time() - start

# Обчислення точності
y_pred = rf_no_pca.predict(X_test)
acc_rf_no_pca = accuracy_score(y_test, y_pred)

print(f"Розмірність даних: {X_train.shape[1]}")
print(f"Час тренування: {time_rf_no_pca:.2f} сек")
print(f"Точність: {acc_rf_no_pca:.4f} ({acc_rf_no_pca*100:.2f}%)")
print("="*70)

Розмірність даних: 784
Час тренування: 3.66 сек
Точність: 0.9691 (96.91%)


я довго думав чому в мене без пса моделька швидше тренується , думаю на n_jobs=-1 бо паралелізація при меншій розмірності більш погано працює або я просто якусь фігню знизу зробив

*Завдання 2: Застосуйте PCA з поясненою дисперсією 95%. Повторіть тренування. Порівняйте час тренування та точніть з попередньою вправою.*

In [11]:
from sklearn.decomposition import PCA

# Застосування PCA
pca = PCA(n_components=0.95, random_state=42)

start = time.time()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
time_pca = time.time() - start

print(f"Час PCA: {time_pca:.2f} сек")
print(f"Оригінальна розмірність: {X_train.shape[1]}")
print(f"Нова розмірність: {X_train_pca.shape[1]}")
print(f"Кількість компонент: {pca.n_components_}")
print(f"Пояснена дисперсія: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Редукція: {(1 - X_train_pca.shape[1]/X_train.shape[1])*100:.1f}%")
print("="*70)

Час PCA: 0.35 сек
Оригінальна розмірність: 784
Нова розмірність: 154
Кількість компонент: 154
Пояснена дисперсія: 0.9502
Редукція: 80.4%


Пояснена дисперсія: 0.9502
це значить що ми зберегли 95% даних, при зменшенні розмірності на 80%. Дууууже круто для величезних датасетів

In [18]:
# Тренування Random Forest на PCA даних
rf_with_pca = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

start = time.time()
rf_with_pca.fit(X_train_pca, y_train)
time_rf_pca = time.time() - start

# Обчислення точності
y_pred_pca = rf_with_pca.predict(X_test_pca)
acc_rf_pca = accuracy_score(y_test, y_pred_pca)

print(f"Розмірність даних: {X_train_pca.shape[1]}")
print(f"Час тренування: {time_rf_pca:.2f} сек")
print(f"Точність: {acc_rf_pca:.4f} ({acc_rf_pca*100:.2f}%)")
print("="*70)

Розмірність даних: 154
Час тренування: 11.66 сек
Точність: 0.9487 (94.87%)


ну от як таке може бути с пса я не розумію але ладно. Точність наче впала як і повинна була. Ну тепер в нас фічі втратились і є кластери. Розмірність точно зменшилася але ЧОМУ воно довше працює

In [23]:
X_test_pca.shape

(10000, 154)

Ну короче я не знаю, нормалізувати дані для рандом форесту наче не дуже треба, я пробував там гірші показники. У висновку я мав написати що пса пришвидшує але походу даних мало для пришвидшення)

А чому в мене з пса довше код раниться(((

*Завдання 3: Повторіть ці ж кроки з логістичною регресією. Зробіть висновки.*

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_no_pca = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)

start_time = time.time()
lr_no_pca.fit(X_train_scaled, y_train)
lr_no_pca_time = time.time() - start_time

y_pred_lr_no_pca = lr_no_pca.predict(X_test_scaled)
lr_no_pca_accuracy = accuracy_score(y_test, y_pred_lr_no_pca)

print(f"Час тренування: {lr_no_pca_time:.2f} сек")
print(f"Точність: {lr_no_pca_accuracy:.4f} ({lr_no_pca_accuracy*100:.2f}%)\n")

Час тренування: 6.22 сек
Точність: 0.9216 (92.16%)



робимо все те саме що і зверху

In [15]:
# Нормалізація PCA даних
scaler_pca = StandardScaler()
X_train_pca_scaled = scaler_pca.fit_transform(X_train_pca)
X_test_pca_scaled = scaler_pca.transform(X_test_pca)

lr_with_pca = LogisticRegression(max_iter=100, random_state=42, n_jobs=-1)

start_time = time.time()
lr_with_pca.fit(X_train_pca_scaled, y_train)
lr_pca_time = time.time() - start_time

y_pred_lr_pca = lr_with_pca.predict(X_test_pca_scaled)
lr_pca_accuracy = accuracy_score(y_test, y_pred_lr_pca)

print(f"Час тренування: {lr_pca_time:.2f} сек")
print(f"Точність: {lr_pca_accuracy:.4f} ({lr_pca_accuracy*100:.2f}%)\n")

Час тренування: 1.47 сек
Точність: 0.9237 (92.37%)



In [24]:
X_test_pca_scaled.shape

(10000, 154)

ну тут все гуд працює і результати як і мають бути з часом тренування

Висновки:

1. Вплив PCA
   - Зменшення розмірності: 784 до 154 компонент (80% редукція)
   - Random Forest: прискорення НЕ відбулося((( але мало бути
   - Logistic Regression: прискорення по часу в 3 рази десь
   - Втрата точності мінімальна (1-2%) для обох випадків

2. Порівняння моделей:
   - Random Forest точніший (~96%)
   - Logistic Regression швидший
   - PCA критично важливий для роботи з величезними датасетами задля прискорення. Але не завжди підходить бо очевидний мінус - втрата фічів.
